In [1]:
import os, base64
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.communication.email import EmailClient

from IPython.display import Image, display

import httpx

load_dotenv()
AZURE_MAPS_KEY = os.environ.get('AZURE_MAPS_KEY')

ImportError: cannot import name 'load_dotenv' from 'dotenv' (/home/devadmin/.pyenv/versions/3.12.8/lib/python3.12/site-packages/dotenv.py)

In [ ]:
width=600
height=400
zoom=13
fieldop_lat=30.4
fieldop_lon=-97.6
aid1_lat=30.5
aid1_lon=-97.5

pin_instances = [
        f"default|co008000|lcFFFFFF||'OP'{fieldop_lon} {fieldop_lat}",
        f"default|coFFFF00|lc000000||'AID'{aid1_lon} {aid1_lat}"
    ]

url = "https://atlas.microsoft.com/map/static"
params = {
    'subscription-key': AZURE_MAPS_KEY,
    'api-version': '2024-04-01',
    'layer': 'basic',
    'style': 'main',
    'zoom': zoom,
    'center': f'{fieldop_lon},{fieldop_lat}',
    'width': width,
    'height': height,
    'pins': pin_instances,
    'path': f'lcFF1493||{fieldop_lon} {fieldop_lat}|{aid1_lon} {aid1_lat}'
}
response = httpx.get(url, params=params)


In [ ]:

image_data = base64.b64encode(response.content).decode('utf-8')

inlineAttachments = [
    {
        "name": "inline-attachment.png",
        "contentId": "my-inline-attachment-1",
        "contentType": "image/png",
        "contentInBase64": image_data
    }
]

In [ ]:
message = {
    "content": {
        "subject": "Welcome to Azure Communication Services Email",
        "plainText": "This email message is sent from Azure Communication Services Email using the Python SDK.",
        "html": "<html><h1>HTML body inline images:</h1><img src=\"cid:my-inline-attachment-1\" /></html>"
    },
    "recipients": {
        "to": [
            {
                "address": "<kris@saveourallies.org>",
                "displayName": "Kris DevOps"
            }
        ]
    },
    "senderAddress": "<donotreply@49647ab0-49e2-40a4-9b30-94ae450c4ac3.azurecomm.net>",
    "attachments": inlineAttachments
}

In [ ]:

connect_string = f"endpoint=https://{settings.MAIL_ENDPOINT}/;accesskey={settings.MAIL_FROM_KEY}"
email_client = EmailClient.from_connection_string(<connection_string>)